In [2]:
import pandas as pd 
import numpy as np 

portfolio = pd.read_excel('./final_exam_data.xlsx',sheet_name='portfolio',index_col=0)
forecasting = pd.read_excel('./final_exam_data.xlsx',sheet_name='forecasting',index_col=0)
fx_carry = pd.read_excel('./final_exam_data.xlsx',sheet_name='fx_carry',index_col=0)
fx_carry.head()

,MXN,SOFR,MXSTR
date,,,
2018-04-03,0.054959,0.000073,0.000298
2018-04-04,0.054852,0.000069,0.000298
2018-04-05,0.055166,0.000069,0.000298
2018-04-06,0.054558,0.000069,0.000298
2018-04-09,0.054871,0.000069,0.000298


## Portfolio

In [38]:
import numpy as np 

def tangency_weights(returns,dropna=True,scale_cov=1, name = 'Tangency'):
    if dropna:
        returns = returns.dropna()

    covmat_full = returns.cov()
    covmat_diag = np.diag(np.diag(covmat_full))
    covmat = scale_cov * covmat_full + (1-scale_cov) * covmat_diag

    weights = np.linalg.solve(covmat,returns.mean())
    weights = weights / weights.sum()
    tangency_weights_df = pd.DataFrame(weights, index=returns.columns, columns=[f'{name} Weights'])

    return tangency_weights_df

def equal_weights(returns): 
    return pd.DataFrame(index = returns.columns, data = 1/len(returns.columns), columns = ['EWP Weights'])

def gmv_weights(returns):
    ones = np.ones(returns.columns.shape)
    cov = returns.cov()* annualization_factor
    cov_inv = np.linalg.inv(cov)
    scaling = 1/(np.transpose(ones) @ cov_inv @ ones)
    gmv_tot = scaling * cov_inv @ ones
    gmv_weights_df = pd.DataFrame(gmv_tot, index=returns.columns, columns=['GMV Weights'])
    return gmv_weights_df

def risk_parity_weights(returns): 
    asset_var = returns.var().to_dict() 
    asset_inv_variance_dict = {asset: 1 / variance for asset, variance in asset_var.items()}
    rp_weights_df = pd.DataFrame.from_dict(asset_inv_variance_dict, orient='index', columns=['RP Weights'])
    return rp_weights_df

def calc_mv_portfolio(mean_rets, cov_matrix, target=None):
    w_tan = tangency_weights(mean_rets)

    if target is None:
        return w_tan

    w_gmv = gmv_weights(mean_rets)
    delta = (target - mean_rets @ w_gmv) / (mean_rets @ w_tan - mean_rets @ w_gmv)
    return delta * w_tan + (1 - delta) * w_gmv

def mv_portfolio_excess_returns(target_ret, ex_ret):
    
    mu_tilde = ex_ret.copy().mean()
    Sigma_adj = ex_ret.copy().cov()
    Sigma_inv = np.linalg.inv(Sigma_adj)
    N = Sigma_adj.shape[0]
    delta_tilde = ((np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde)) * target_ret
    tan_wts = tangency_weights(ex_ret)
    omega_star = delta_tilde * tan_wts
    
    return omega_star
    

def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, scale_cov = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, scale_cov = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    MV = pd.DataFrame(index = tot_returns.columns, data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, scale_cov = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values

    return MV



In [19]:
# normalize returns to his the target

target = 0.0025
tan_weights = tangency_weights(data)
adjustment_factor = target / (data.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
tan_weights

NameError: name 'data' is not defined

# Q1

In [20]:
target = 0.0025
tan_weights = tangency_weights(portfolio)
adjustment_factor = target / (portfolio.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
tan_weights

,Tangency Weights
SPY,0.479969
BTC,0.079311
USO,-0.024120
TLT,-0.022660
IEF,0.093716
IYR,-0.246719
GLD,0.200172


In [21]:
tangency_weights(portfolio)

,Tangency Weights
SPY,0.857595
BTC,0.141710
USO,-0.043098
TLT,-0.040488
IEF,0.167449
IYR,-0.440830
GLD,0.357662


(b)

In [45]:
annualization_factor = 52
gmv_weights(portfolio)

,GMV Weights
SPY,0.092076
BTC,-0.001009
USO,0.002750
TLT,-0.476953
IEF,1.427028
IYR,-0.041140
GLD,-0.002752


In [37]:
    mv_portfolio_excess_returns(0.0025, portfolio)

,Tangency Weights
SPY,0.479969
BTC,0.079311
USO,-0.024120
TLT,-0.022660
IEF,0.093716
IYR,-0.246719
GLD,0.200172


In [30]:
def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, scale_cov = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, scale_cov = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    MV = pd.DataFrame(index = tot_returns.columns, data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, scale_cov = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values

    return MV
    
mv_portfolio(0.0025, portfolio)


,MV Weights,tangency weights,GMV weights
SPY,0.498982,0.857595,0.092076
BTC,0.074852,0.141710,-0.001009
USO,-0.021620,-0.043098,0.002750
TLT,-0.244953,-0.040488,-0.476953
IEF,0.757509,0.167449,1.427028
IYR,-0.253592,-0.440830,-0.041140
GLD,0.188823,0.357662,-0.002752


(c)
- part 1 involves excess returns which is net of risk free rates while part 2 involves totla returns.
- Excess Returns Approach:

Takes into account the presence of a risk-free asset, which shifts the efficient frontier by incorporating the risk-free rate.
The tangency portfolio includes leverage if the Sharpe ratio is high, assuming the investor can borrow/lend at the risk-free rate.

- Total Returns Approach:

Assumes no risk-free rate is available, or it is ignored for simplicity.
The GMV portfolio minimizes total portfolio risk, regardless of return.
The MV portfolio with a target return focuses solely on absolute returns, without considering excess return over a risk-free rate.
 

In [41]:
# (d)

portfolio_copy = portfolio.drop(columns = ['BTC'])
tangency_weights(portfolio_copy)

,Tangency Weights
SPY,0.975792
USO,-0.034911
TLT,0.012283
IEF,0.071743
IYR,-0.483277
GLD,0.458370


In [84]:
in_sample = portfolio_copy.loc[:'2021',:]
oo_sample = portfolio_copy.loc['2022':'2023':]


In [91]:
target = 0.0025
tan_weights = tangency_weights(in_sample)
adjustment_factor = target / (in_sample.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
is_weights = tan_weights
is_weights


,Tangency Weights
SPY,0.708230
USO,-0.034790
TLT,-0.279598
IEF,1.712950
IYR,-0.252887
GLD,0.023858


In [92]:
oos_returns = oo_sample @ is_weights


In [93]:
def portfolio_metrics_basic(returns: pd.DataFrame, annualization_factor: float) -> pd.DataFrame: 
    '''
    Returns Annualized ER, Vol, SR, 
    '''
    
    returns_df = pd.DataFrame(index = returns.columns) 
    mu = returns.mean() * annualization_factor
    vol = returns.std() * np.sqrt(annualization_factor)
    sr = mu / vol

    
    returns_df['Annnualized Return'] = mu.values 
    returns_df['Annnualized Vol'] = vol.values 
    returns_df['Annnualized SR'] = sr.values
    
    return returns_df


In [94]:
portfolio_metrics_basic(oos_returns, annualization_factor)


,Annnualized Return,Annnualized Vol,Annnualized SR
Tangency Weights,-0.07043,0.163527,-0.430691


In [95]:
ewp = equal_weights(in_sample)
adjustment_factor = target / (in_sample.mean() @ ewp.values)
ewp *= adjustment_factor
oos_weights = ewp
oos_returns = oo_sample @ ewp
portfolio_metrics_basic(oos_returns, annualization_factor)

,Annnualized Return,Annnualized Vol,Annnualized SR
EWP Weights,-0.036569,0.183591,-0.199185


The EWP portfolio performed better with an improved annualized SR and higher expected return, despite experiencing a higher volatility as ell. 

# Q3

In [96]:
forecasting

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,USO,Tnote rate,Tnote rate change
date,,,
2009-05-31,0.271394,3.465,0.341
2009-06-30,0.042033,3.523,0.058
2009-07-31,-0.029528,3.501,-0.022
2009-08-31,-0.020647,3.401,-0.100
2009-09-30,0.003883,3.307,-0.094
...,...,...,...
2023-07-31,0.151377,3.959,0.140
2023-08-31,0.025830,4.093,0.134
2023-09-30,0.077272,4.573,0.480


In [104]:
forecast_copy = forecasting.copy() 
forecast_copy[['Tnote rate shifted', 'Tnote rate change shifted']] = forecasting[['Tnote rate', 'Tnote rate change']].shift()
forecast_copy = forecast_copy.dropna()
import statsmodels.api as sm 
y = forecast_copy['USO'] 
x = forecast_copy[['Tnote rate shifted', 'Tnote rate change shifted']] 
model = sm.OLS(y,sm.add_constant(x)).fit()
model.rsquared

(0.02813716766965546,
 const                        0.020120
 Tnote rate shifted          -0.009569
 Tnote rate change shifted    0.074061
 dtype: float64)

In [253]:
def time_series_regression(portfolio, factors, alpha = True, multiple_factors = False, resid = False, annualization = 12):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    if alpha: 
        rhs = sm.add_constant(factors)
    else: 
        rhs = factors

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const']*annualization
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
        else:
            print(res.params[1])
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] =  annualization* portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

In [231]:
USO = forecasting.loc[:,['USO']]
signal = ['Tnote rate','Tnote rate change']
factors = forecasting.loc[:,['Tnote rate','Tnote rate change']].shift(1)
signal_reg = time_series_regression(USO, factors, alpha = True, multiple_factors=True, resid=False)
signal_reg


            Tnote rate  Tnote rate change
date                                     
2009-05-31         NaN                NaN
2009-06-30       3.465              0.341
2009-07-31       3.523              0.058
2009-08-31       3.501             -0.022
2009-09-30       3.401             -0.100
...                ...                ...
2023-07-31       3.819              0.182
2023-08-31       3.959              0.140
2023-09-30       4.093              0.134
2023-10-31       4.573              0.480
2023-11-30       4.875              0.302

[175 rows x 2 columns] const                0.020120
Tnote rate          -0.009569
Tnote rate change    0.074061
dtype: float64


,alpha_hat,Tnote rate beta,Tnote rate change beta,info_ratio,treynor_ratio,R-squared,Tracking Error
USO,0.241445,-0.009569,0.074061,0.687085,0.440459,0.028137,0.351405


In [232]:
forecasted_rets = (np.array(forecasting.shift(1).loc[:,['Tnote rate','Tnote rate change']]) 
                   @ np.array(signal_reg.loc[:,['Tnote rate beta','Tnote rate change beta']].T))

forecast_ret = (pd.DataFrame(forecasted_rets,columns = ['Forecasted Return'],index= forecasting.index)) 

forecast_ret['Forecasted Return'] = (forecast_ret['Forecasted Return'] + 
                                     float(signal_reg['alpha_hat']))*50 + 0.5

strat = pd.DataFrame(forecast_ret['Forecasted Return'] * forecasting.loc[:,['USO']]['USO'], 
                     columns=forecast_ret.columns, 
                     index=forecast_ret.index).dropna()
strat

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2490/1586364897.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(signal_reg['alpha_hat']))*50 + 0.5


,Forecasted Return
date,
2009-06-30,0.511843
2009-07-31,-0.327805
2009-08-31,-0.223309
2009-09-30,0.041067
2009-10-31,0.920401
...,...
2023-07-31,1.728577
2023-08-31,0.289209
2023-09-30,0.858502


In [233]:
portfolio_metrics_basic(strat, 12)

,Annnualized Return,Annnualized Vol,Annnualized SR
Forecasted Return,-0.085586,4.039307,-0.021188


In [254]:
# amount of USO relative to the returns 
x = forecasting.loc[:,['USO']].iloc[1:]
y = strat
reg = time_series_regression(y,x, alpha = True, multiple_factors=False, resid=False)
reg

11.301633856800688


,alpha_hat,USO beta,info_ratio,treynor_ratio,R-squared,Tracking Error
Forecasted Return,0.173851,11.301634,0.589537,-0.007573,0.99467,0.294893


In [252]:
beta = reg.loc['Forecasted Return', 'USO beta']
hedged = beta * x 
returns = hedged['USO'] + y['Forecasted Return']
mu = returns.mean() * annualization_factor
vol = returns.std() * np.sqrt(annualization_factor)
sr = mu / vol
mu, vol, sr

(-7.245494201693663, 36.94679530278684, -0.19610616136840284)

- Hedge Ratio = 11.301
- 

# Q4

In [137]:
fx_carry.columns

Index(['MXN', 'SOFR', 'MXSTR'], dtype='object')

In [138]:
fx_carry['log_MXN'] = np.log(fx_carry['MXN'])
fx_carry['log_SOFR'] = np.log(1+fx_carry['SOFR'])
fx_carry['log_MXSTR'] = np.log(1+fx_carry['MXSTR'])
fx_carry.head()

,MXN,SOFR,MXSTR,log_MXN,log_SOFR,log_MXSTR
date,,,,,,
2018-04-03,0.054959,0.000073,0.000298,-2.901174,0.000073,0.000298
2018-04-04,0.054852,0.000069,0.000298,-2.903123,0.000069,0.000298
2018-04-05,0.055166,0.000069,0.000298,-2.897403,0.000069,0.000298
2018-04-06,0.054558,0.000069,0.000298,-2.908485,0.000069,0.000298
2018-04-09,0.054871,0.000069,0.000298,-2.902767,0.000069,0.000298


In [148]:
fx_carry['log_MXN'].mean()

-2.981326252448035

since these are USD per MXN, to get the log return of holding MXN, -> mxn rate - sofr rate

In [153]:
returns = fx_carry['log_MXN'].diff() + fx_carry['log_MXSTR'].shift()  - fx_carry['log_SOFR'].shift()
annualization_factor = 252 

mu = returns.mean() * annualization_factor
vol = returns.std() * np.sqrt(annualization_factor)
sr = mu / vol
mu, vol, sr
returns

date
2018-04-03         NaN
2018-04-04   -0.001724
2018-04-05    0.005949
2018-04-06   -0.010854
2018-04-09    0.005946
                ...   
2023-11-24    0.004658
2023-11-27   -0.001311
2023-11-28    0.002219
2023-11-29   -0.004672
2023-11-30   -0.009806
Length: 1380, dtype: float64

In [158]:
fx_carry['log_MXSTR'].shift()  - fx_carry['log_SOFR'].shift()

date
2018-04-03         NaN
2018-04-04    0.000225
2018-04-05    0.000229
2018-04-06    0.000228
2018-04-09    0.000228
                ...   
2023-11-24    0.000236
2023-11-27    0.000235
2023-11-28    0.000235
2023-11-29    0.000235
2023-11-30    0.000236
Length: 1380, dtype: float64

It was better to long MXN relative to USD as the returns cumulatively increased. The interest spread helped. 
- USD depreciated relative to MXN as the amount of USD per MXN increased and the returns was negative 

In [155]:
fx_data = fx_carry.copy() 
log_fx_data = np.log(1 + fx_data)
log_fx_data['MXN'] = np.log(fx_data['MXN']).fillna(method='ffill')
log_fx_data.mean().to_frame("Mean").T
log_fx_data['MXN_diff'] = log_fx_data['MXN'].diff().shift(-1)
excess_log_ret = log_fx_data['MXN_diff'] - (log_fx_data['SOFR'] - log_fx_data['MXSTR'])
mu = excess_log_ret.mean() * annualization_factor
vol = excess_log_ret.std() * np.sqrt(annualization_factor)
sr = mu / vol
mu, vol, sr

/Users/patsy/anaconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


(0.06351034946222583, 0.1279733380976997, 0.49627797794677847)

In [47]:
fx_spot_list = ['log_GBP'
                ,'log_EUR'
                ,'log_CHF'
                ,'log_JPY']

fx_hldg_lst = []
for fx in fx_spot_list:
    print(fx)
    # this basically compares the change in spot fx rate + rf of x (of the previous day) - rf usd (of the previous day)
    fx_hldg_excess_ret = fx_rates[fx] - fx_rates[fx].shift(1) + risk_free_rates[fx].shift(1) - risk_free_rates['log_USD'].shift(1)
    fx_hldg_summary = performance_summary(fx_hldg_excess_ret.to_frame().dropna(), 365) #the performance metrics 
    fx_hldg_lst.append(fx_hldg_summary)

fx_hldg_perf_summary = pd.concat(fx_hldg_lst)
perf = fx_hldg_perf_summary.T
perf.columns = fx_spot_list 
perf

log_GBP


NameError: name 'performance_summary' is not defined